In [1]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

rs = 0

In [13]:
feature_dfs = [pd.read_csv(f'output/feat_df_{str(i)}').drop('Unnamed: 0', axis=1) for i in range(6)]
combined_feats = pd.read_csv('output/combined_feats').drop('Unnamed: 0', axis=1)
train_scores = pd.read_csv('input/train_scores.csv')

In [17]:
X, y = combined_feats, train_scores['score']
X, X_test, y, y_test = train_test_split(X, y, test_size=.2, random_state=rs)
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=.25, random_state=rs)
df_names = ['pattern info', 'n-gram tdidf', 'iki info', 'agg info', 'other count info', 'gap info']
y_trains = []
y_valids = []
for i, df in enumerate(feature_dfs):
    model = GradientBoostingRegressor()
    model.fit(X_train[df.columns], y_train)
    y_predict_train = model.predict(X_train[df.columns])
    y_predict_valid = model.predict(X_valid[df.columns])
    # print(f'MSQE: training set for df')
    # print(mean_squared_error(y_train, y_predict_train, squared=False))
    print(f'MSQE: validation set for {df_names[i]}')
    print(round(mean_squared_error(y_valid, y_predict_valid, squared=False), 3))
    y_trains.append(y_predict_train)
    y_valids.append(y_predict_valid)

MSQE: validation set for pattern info
0.82
MSQE: validation set for n-gram tdidf
0.702
MSQE: validation set for iki info
0.725
MSQE: validation set for agg info
0.656
MSQE: validation set for other count info
0.727
MSQE: validation set for gap info
0.683


In [22]:
weights = [.1, .1, .1, .35, .1, .25]
y_train_combined, y_valids_combined = y_trains[0] * weights[0], y_valids[0] * weights[0]
for i in range(1, len(weights)):
    y_train_combined += y_trains[i] * weights[i]
    y_valids_combined += y_valids[i] * weights[i]
print(f'MSQE: training set')
print(round(mean_squared_error(y_train_combined, y_predict_train, squared=False), 3))
print(f'MSQE: validation set for {df_names[i]}')
print(round(mean_squared_error(y_valids_combined, y_predict_valid, squared=False), 3))

MSQE: training set
0.191
MSQE: validation set for gap info
0.232
